# Outline
We’ll start with an example that does a “cone search”; that is, it selects stars that appear in a circular region of the sky.

Then, to select stars in the vicinity of GD-1, we will:

Use Quantity objects to represent measurements with units.

Use Astropy to convert coordinates from one frame to another.

Use the ADQL keywords POLYGON, CONTAINS, and POINT to select stars that fall within a polygonal region.

Submit a query and download the results.

Store the results in a FITS file.

In [4]:
import astropy.units as u
from astroquery.gaia import Gaia

# Selecting a Region
Documentaion: https://gea.esac.esa.int/archive-help/adql/examples/index.html

Example:

In [2]:
cone_query = """SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

This query uses three keywords that are specific to ADQL (not SQL):

**POINT**: a location in ICRS coordinates, specified in degrees of right ascension and declination.

**CIRCLE**: a circle where the first two values are the coordinates of the center and the third is the radius in degrees.

**CONTAINS**: a function that returns 1 if a POINT is contained in a shape and 0 otherwise.

In [5]:
cone_job=Gaia.launch_job(cone_query)
cone_job

In [7]:
results=cone_job.get_results()
results

source_id
int64
3322773965056065536
3322773758899157120
3322774068134271104
3322773930696320512
3322774377374425728
3322773724537891456
3322773724537891328
3322773930696321792
3322773724537890944


# Getting GD-1 Data
Below we show the steps to go from Equatorial coordinates (sky coordinates) to Galactic coordinates and finally to a reference frame defined to more easily study GD-1.

In [8]:
from astropy.coordinates import SkyCoord

### Defining Region

In [10]:
ra= 88.8 *u.degree
dec=7.4 *u.degree
coord_icrs=SkyCoord(ra=ra, dec=dec, frame='icrs')

`SkyCoord` objects require units in order to understand the context. There are a number of ways to define `SkyCoord` objects, in our example, we explicitly specified the coordinates and units and provided a reference frame.

`SkyCoord` provides the `transform_to` function to transform from one reference frame to another reference frame. For example, we can transform `coords_icrs` to Galactic coordinates like this:

### Transforming Coordinates

In [12]:
coord_galactic=coord_icrs.transform_to('galactic')
coord_galactic

<SkyCoord (Galactic): (l, b) in deg
    (199.79693102, -8.95591653)>

*Notice that in the Galactic frame, the coordinates are the variables we usually use for Galactic longitude and latitude called `l` and `b`, respectively, not `ra` and `dec`. Most reference frames have different ways to specify coordinates and the `SkyCoord` object will use these names.*

To transform to and from GD-1 coordinates, we will use a frame defined by `Gala`, which is an Astropy-affiliated library that provides tools for galactic dynamics.

Gala provides GD1Koposov10, which is “a Heliocentric spherical coordinate system defined by the orbit of the GD-1 stream”.\
In this coordinate system, one axis is defined along the direction of the steam (the x-axis in Figure 1) and one axis is defined perpendicular to the direction of the stream (the y-axis in Figure 1). These are called the $\phi_{1}$ and $\phi_{2}$ coordinates, respectively.

In [13]:
from gala.coordinates import GD1Koposov10

gd1_frame=GD1Koposov10()
gd1_frame

<GD1Koposov10 Frame>

We can use it to find the coordinates of Betelgeuse in the GD-1 frame, like this:

In [14]:
coord_galactic=coord_icrs.transform_to(gd1_frame)
coord_galactic

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    (-94.97222038, 34.5813813)>

# Selecting a Rectangle

Now that we know how to define coordinate transformations, we are going to use them to get a list of stars that are in GD-1. As we mentioned before, this is a lot of stars, so we are going to start by defining a rectangle that encompasses a small part of GD-1. This is easiest to define in GD-1 coordinates.

### Define a rectangluar region

In [15]:
phi1_min = -55 * u.degree 
phi1_max = -45 * u.degree
phi2_min = -8 * u.degree
phi2_max = 4 * u.degree

#### Function to create a rectangle

In [16]:
def make_rectangle(x1, x2, y1, y2):
    """Return the corners of a rectangle."""
    xs = [x1, x1, x2, x2, x1]
    ys = [y1, y2, y2, y1, y1]
    return xs, ys

In [17]:
phi1_rect, phi2_rect = make_rectangle(
    phi1_min, phi1_max, phi2_min, phi2_max)

`phi1_rect` and `phi2_rect` contains the coordinates of the corners of a rectangle in the GD-1 frame.

While it is easier to visualize the regions we want to define in the GD-1 frame, the coordinates in the Gaia catalog are in the ICRS frame. \
In order to use the rectangle we defined, we need to convert the coordinates from the GD-1 frame to the ICRS frame. We will do this using the SkyCoord object. Fortunately SkyCoord objects can take lists of coordinates, in addition to single values.

In [18]:
corners=SkyCoord(phi1=phi1_rect, phi2=phi2_rect, frame=gd1_frame)
corners

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.), (-55., -8.)]>

In [24]:
corners_icrs=corners.transform_to('icrs')
corners_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254),
     (146.27533314, 19.26190982)]>

*Notice that a rectangle in one coordinate system is not necessarily a rectangle in another. In this example, the result is a (non-rectangular) polygon. This is why we defined our rectangle in the GD-1 frame.*

# Converting to Strings

In order to use this polygon as part of an ADQL query, we have to convert it to a string with a comma-separated list of coordinates

In [22]:
# Here we define a function to convert a one-dimenstional list of SkyCoord to string for Gaia's query format.
def skycoord_to_string(skycoord):
    """Convert a one-dimenstional list of SkyCoord to string for Gaia's query format."""
    corners_list_str = skycoord.to_string()
    corners_single_str = ' '.join(corners_list_str)
    return corners_single_str.replace(' ', ', ')

In [25]:
sky_point_list = skycoord_to_string(corners_icrs)
sky_point_list

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

# Assembling the query


Now we are ready to assemble our query to get all of the stars in the Gaia catalog that are in the small rectangle we defined and are likely to be part of GD-1 with the criteria we previously defined.

In [26]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'

In [31]:
polygon_query_base="""
SELECT
{columns}
FROM gaiadr2.gaia_source
WHERE parallax <1
AND bp_rp BETWEEN -0.75 AND 2
AND 1=CONTAINS(POINT(ra, dec),
POLYGON({sky_point_list}))
"""

In [32]:
polygon_query=polygon_query_base.format(columns=columns, sky_point_list=sky_point_list)
print(polygon_query)


SELECT
source_id, ra, dec, pmra, pmdec, parallax
FROM gaiadr2.gaia_source
WHERE parallax <1
AND bp_rp BETWEEN -0.75 AND 2
AND 1=CONTAINS(POINT(ra, dec),
POLYGON(146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619))



In [33]:
polygon_job=Gaia.launch_job_async(polygon_query)
print(polygon_job)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140339>
   name    dtype    unit                              description                            
--------- ------- -------- ------------------------------------------------------------------
source_id   int64          Unique source identifier (unique within a particular Data Release)
       ra float64      deg                                                    Right ascension
      dec float64      deg                                                        Declination
     pmra float64 mas / yr                         Proper motion in right ascension direction
    pmdec float64 mas / yr                             Proper motion in declination direction
 parallax float64      mas                                                           Parallax
Jobid: 1685734679233O
Phase: COMPLETED
Owner: None
Output file: async_20230603010759.vot
Results: None


In [34]:
polygon_results=polygon_job.get_results()
len(polygon_results)

140339

# Saving Results

In [35]:
filename="gd1_results.fits"
polygon_results.write(filename, overwrite=True)

#### To know the file size

In [36]:
from os.path import getsize

MB = 1024 * 1024
getsize(filename) / MB

6.4324951171875